In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


# Install

In [ ]:
#@markdown Install dependencies. You only need to run this once.
!wget -q -O train_dreambooth.py https://github.com/geocine/smd-diffusion/raw/main/train_dreambooth.py
!wget -q -O convert_diffusers_to_original_stable_diffusion.py https://github.com/geocine/smd-diffusion/raw/main/convert_diffusers_to_original_stable_diffusion.py
!wget -q -O concepts_list.json https://github.com/geocine/smd-diffusion/raw/main/concepts_list.json
#@markdown Install dependencies. You don't need to change any of these settings.
!wget -q -O train_dreambooth.py https://github.com/geocine/smd-diffusion/raw/main/train_dreambooth.py
!wget -q -O convert_diffusers_to_original_stable_diffusion.py https://github.com/geocine/smd-diffusion/raw/main/convert_diffusers_to_original_stable_diffusion.py
!wget -q -O concepts_list.json https://github.com/geocine/smd-diffusion/raw/main/concepts_list.json
# URLs of the diffusers and xformers packages
DIFFUSERS_URL = 'git+https://github.com/ShivamShrirao/diffusers' #@param {type:"string"}
XFORMERS_URL = 'https://github.com/geocine/dreamstall-binaries/releases/download/cxx-p38-txx-linux/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl' #@param {type:"string"}
FORCE_REINSTALL = False #@param {type:"boolean"}
def install_package(package, force_reinstall=False):
    if package == 'diffusers':
        # Check if the package is already installed using pip freeze
        installed_packages = !pip freeze
        if not force_reinstall and any(package in s for s in installed_packages):
            print(f'{package} is already installed')
            return
        else:
            # Install the package using the URL
            !pip install -qq {DIFFUSERS_URL}
    elif package == 'xformers':
        # Check if the package is already installed using pip freeze
        installed_packages = !pip freeze
        if not force_reinstall and any(package in s for s in installed_packages):
            print(f'{package} is already installed')
            return
        else:
            # Install the package using the URL
            !pip install -qq {XFORMERS_URL}
    else:
        # Check if the package is already installed using pip freeze
        installed_packages = !pip freeze
        if not force_reinstall and any(package in s for s in installed_packages):
            print(f'{package} is already installed')
            return
        else:
            # Install the package using pip
            !pip install -q {package}


# List of packages to check and install
packages = ['diffusers', 'triton', 'accelerate==0.12.0', 'transformers', 'ftfy', 'bitsandbytes', 'gradio', 'natsort', 'xformers']

# Check and install each package
for package in packages:
    install_package(package, FORCE_REINSTALL)

## Settings


<details>
  <summary>Details about <code>{SDD_TOKEN}</code> and <code>{SDD_CLASS}</code></summary>

  - `SDD_TOKEN` - corresponds to the unique identifier which will reference the subject we want to add. This name should be unique, so we don’t have to compete with an existing representation
  - `SDD_CLASS` - use generic classes such as man, woman, or child (if the subject is a person) or cat or dog (if the subject is a pet)

  > You could explore other classes. In this Colab I use the class `supermodel` by default since I get good results in training my personal subjects
</details>

In [ ]:
import os
import json

SDD_TOKEN = "zwx" #@param {type:"string"}
SDD_CLASS = "supermodel" #@param {type:"string"}


# Open the JSON file and read the contents
with open("/content/concepts_list.json", "r") as f:
  json_data = json.load(f)

# Iterate over the object and replace the placeholders with the values
for item in json_data:
  for key, value in item.items():
    item[key] = value.format(SDD_TOKEN=SDD_TOKEN, SDD_CLASS=SDD_CLASS)

# Open the JSON file and write the updated contents
with open("/content/concepts_list.json", "w") as f:
  json.dump(json_data, f, indent=2)

#@markdown You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an [access token](https://huggingface.co/settings/tokens) for the code to work.
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

#@markdown Name/Path of the initial model.
MODEL_NAME = "runwayml/stable-diffusion-v1-5" #@param {type:"string"}

OUTPUT_DIR = f"stable_diffusion_models/{SDD_TOKEN}"
OUTPUT_DIR = "/content/" + OUTPUT_DIR

if os.path.exists(OUTPUT_DIR):
  # Remove all files inside the directory
  for file in os.listdir(OUTPUT_DIR):
    os.remove(os.path.join(OUTPUT_DIR, file))
else:
  # Create the directory
  os.makedirs(OUTPUT_DIR)

print(f"[*] Models will be saved at {OUTPUT_DIR}")

#Unzipping the regularization images
from huggingface_hub import hf_hub_download
import subprocess
zip_file = hf_hub_download(
    repo_id="geocine/regularization-images", 
    filename=f"{SDD_CLASS}_v1-5_mse_vae_ddim50_cfg7_n4411.zip", 
    revision="main",
    repo_type="dataset"
)

unzip_directory = f"/content/data/{SDD_CLASS}"

# Check if the unzip directory exists
try:
  # Get a list of the files in the unzip directory
  files = os.listdir(unzip_directory)
except FileNotFoundError:
  # Create the unzip directory
  os.makedirs(unzip_directory)
  # Set the files list to an empty list
  files = []

# Check if the unzip directory has files
if len(files) > 0:
  # Do not run the unzip command
  print("Unzip directory has files. Skipping unzip.")
else:
  # Run the unzip command
  subprocess.run(["unzip", "-j", zip_file, "-d", unzip_directory])

# Train

In [ ]:
#@markdown Upload your images by running this cell.

import os
import json
from google.colab import files
import shutil

# Load the data from the JSON file into the concepts_list variable
with open("/content/concepts_list.json", "r") as f:
    concepts_list = json.load(f)


# Incorporate this so that users won't have to crop their images https://github.com/d8ahazard/sd_smartprocess
for c in concepts_list:
   print(f"Uploading instance images for `{c['instance_prompt']}`")
   uploaded = files.upload()
   for filename in uploaded.keys():
       dst_path = os.path.join(c['instance_data_dir'], filename)
       # Create the instance_data_dir directory if it does not exist
       os.makedirs(c['instance_data_dir'], exist_ok=True)
       shutil.move(filename, dst_path)

In [ ]:
NUM_CLASS_IMAGES = 3000 #@param {type:"number"}
#@markdown `{TOKEN_CLASS}` will be replaced with the token and class name.
SAVE_SAMPLE_PROMPT = "photo of {TOKEN_CLASS}" #@param {type:"string"}
SAVE_SAMPLE_PROMPT = SAVE_SAMPLE_PROMPT.format(TOKEN_CLASS=f"{SDD_TOKEN} {SDD_CLASS}")
MAX_TRAIN_STEPS = 3000 #@param {type:"number"}
SAVE_INTERVAL = 400 #@param {type:"number"}
SAVE_MIN_STEPS = 2000 #@param {type:"number"}
CLEAR_MODELS = True #@param {type:"boolean"}

#@markdown If you have experience with training models, you can change more parameters on the code in this cell.

PREV_MODEL_STEPS = None
g_cuda = None

if CLEAR_MODELS:
    # Run the rm command using subprocess
    subprocess.run(["rm", "-rf", f"/content/stable_diffusion_models/{SDD_TOKEN}/*"])

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="main" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=$NUM_CLASS_IMAGES \
  --sample_batch_size=4 \
  --max_train_steps=$MAX_TRAIN_STEPS \
  --save_interval=$SAVE_INTERVAL \
  --save_min_steps=$SAVE_MIN_STEPS \
  --save_sample_prompt="$SAVE_SAMPLE_PROMPT" \
  --concepts_list="concepts_list.json"
# --shuffle_after_epoch

In [ ]:
#@markdown Run to generate a grid of preview images from the last saved models.
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

models_folder = OUTPUT_DIR
folders = sorted([f for f in os.listdir(models_folder) if f != "0"], key=lambda x: int(x))

row = len(folders)
col = len(os.listdir(os.path.join(models_folder, folders[0], "samples")))
scale = 4
fig, axes = plt.subplots(row, col, figsize=(col*scale, row*scale), gridspec_kw={'hspace': 0, 'wspace': 0})

for i, folder in enumerate(folders):
    folder_path = os.path.join(models_folder, folder)
    image_folder = os.path.join(folder_path, "samples")
    images = [f for f in os.listdir(image_folder)]
    for j, image in enumerate(images):
        if row == 1:
            currAxes = axes[j]
        else:
            currAxes = axes[i, j]
        if i == 0:
            currAxes.set_title(f"Image {j}")
        if j == 0:
            currAxes.text(-0.1, 0.5, folder, rotation=0, va='center', ha='center', transform=currAxes.transAxes)
        image_path = os.path.join(image_folder, image)
        img = mpimg.imread(image_path)
        currAxes.imshow(img, cmap='gray')
        currAxes.axis('off')
        
plt.tight_layout()
plt.savefig('grid.png', dpi=72)

# Generate

In [ ]:
import random 
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler, EulerAncestralDiscreteScheduler
from IPython.display import display

MODEL_STEPS = 2000 #@param {type:"number"}

if MODEL_STEPS != PREV_MODEL_STEPS or g_cuda is None:
  PREV_MODEL_STEPS = MODEL_STEPS
  model_path = f'/content/stable_diffusion_models/{{TOKEN}}/{MODEL_STEPS}'             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive
  model_path = model_path.replace("{TOKEN}", SDD_TOKEN)
  scheduler = EulerAncestralDiscreteScheduler(num_train_timesteps=1000, beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
  pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")
  pipe.enable_xformers_memory_efficient_attention()
  g_cuda = torch.Generator(device='cuda')

SEED = -1  #@param {type:"number"}
if (SEED < 0):
  SEED = random.randint(0, 2**32 - 1) 
g_cuda.manual_seed(SEED)

#@markdown Enter a prompt to generate images from. `{TOKEN_CLASS}` will be replaced with the token and class name.
PROMPT = "photo of {TOKEN_CLASS}" #@param {type:"string"}
PROMPT = PROMPT.format(TOKEN_CLASS=f"{SDD_TOKEN} {SDD_CLASS}")
NEGATIVE_PROMPT = "" #@param {type:"string"}
NUM_SAMPLES = 2 #@param {type:"number"}
CFG = 8 #@param {type:"number"}
STEPS = 80 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt=PROMPT,
        height=height,
        width=width,
        negative_prompt=NEGATIVE_PROMPT,
        num_images_per_prompt=NUM_SAMPLES,
        num_inference_steps=STEPS,
        guidance_scale=CFG,
        generator=g_cuda
    ).images

for img in images:
    display(img)

# Save

In [ ]:
import os
from google.colab import drive

#@markdown This will save chosen checkpoint on Google Drive. You can then download it from there.
MODEL_STEPS = 2800 #@param {type:"number"}
mdl_path = f"/content/stable_diffusion_models/{{TOKEN}}/{MODEL_STEPS}"
mdl_path = mdl_path.replace("{TOKEN}", SDD_TOKEN)
ckpt_path =  mdl_path + "/model.ckpt"

!python convert_diffusers_to_original_stable_diffusion.py --model_path $mdl_path  --checkpoint_path $ckpt_path --half
print(f"[*] Converted ckpt saved at {ckpt_path}")

# Check if Google Drive is already mounted
if not os.path.exists("/content/drive"):
    # Mount Google Drive
    drive.mount("/content/drive")

NAME = "aivan" #@param {type:"string"}
#@markdown Enter the path to save the model in Google Drive. If left empty, the model will be saved in the root of Google Drive.
GDRIVE_PATH = "Files" #@param {type:"string"}

# remove / from start and end of GDRIVE_PATH if they exist
GDRIVE_PATH = GDRIVE_PATH.strip('/')
MODEL_NAME = f"{SDD_CLASS}-{MODEL_STEPS}-{SDD_TOKEN}-{NAME}"
if GDRIVE_PATH:
    cmd = f"cp /content/stable_diffusion_models/{SDD_TOKEN}/{MODEL_STEPS}/model.ckpt /content/drive/MyDrive/{GDRIVE_PATH}/{MODEL_NAME}.ckpt"
else:
    cmd = f"cp /content/stable_diffusion_models/{SDD_TOKEN}/{MODEL_STEPS}/model.ckpt /content/drive/MyDrive/{MODEL_NAME}.ckpt"

# Execute the command
!{cmd}
print(f"Model saved at /{GDRIVE_PATH}/{MODEL_NAME}. Wait for 5 minutes before closing")
print(f"To use your model on other applications make sure to mention \"{SDD_TOKEN} {SDD_CLASS}\" in the prompt.")

In [ ]:
#@title Free runtime memory
exit()